In [1]:
import sys
from PyQt5.QtWidgets import *
from PyQt5.QAxContainer import *
from PyQt5.QtCore import *
from PyQt5 import QtTest
from datetime import datetime
import json
import os.path

In [2]:
SEARCH_DATE = datetime.now().strftime('%Y_%m_%d') #파일 저장용 검색 날짜
COM_CODE = "000660" # SK하이닉스
COM_DATE = datetime.now().strftime('%Y%m%d') #검색날짜

In [3]:
output_list = {
    'OPT10001': ['종목코드',
                 '종목명',
                 '결산월',
                 '액면가',
                 '자본금',
                 '상장주식',
                 '신용비율',
                 '연중최고',
                 '시가총액',
                 '시가총액비중',
                 '외인소진률',
                 '대용가'
                 'PER',
                 'EPS',
                 'ROE',
                 'PBR',
                 'EV',
                 'BPS',
                 '매출액',
                 '영업이익',
                 '당기순이익',
                 '250최고',
                 '250최저',
                 '시가',
                 '고가',
                 '저가',
                 '상한가',
                 '하한가',
                 '기준가',
                 '예상체결가',
                 '예상체결수량',
                 '250최고가일',
                 '250최고가대비율',
                 '250최저가일',
                 '250최저가대비율',
                 '현재가',
                 '대비기호',
                 '전일대비',
                 '등락율',
                 '거래량',
                 '거래대비',
                 '액면가단위',
                 '유통주식',
                 '유통비율',
                 ],
    'OPT10004': ['호가잔량기준시간',
                 '매도10차선잔량',
                 '매도10차선호가',
                 '매도9차선잔량',
                 '매도9차선호가',
                 '매도8차선잔량',
                 '매도8차선호가',
                 '매도7차선잔량',
                 '매도7차선호가',
                 '매도6차선잔량',
                 '매도6차선호가',
                 '매도5차선잔량',
                 '매도5차선호가',
                 '매도4차선잔량',
                 '매도4차선호가',
                 '매도3차선잔량',
                 '매도3차선호가',
                 '매도2차선잔량',
                 '매도2차선호가',
                 '매도최우선잔량',
                 '매도최우선호가',
                 '매수최우선호가',
                 '매수최우선잔량',
                 '매도2차선호가',
                 '매도2차선잔량',
                 '매도3차선호가',
                 '매도3차선잔량',
                 '매도4차선호가',
                 '매도4차선잔량',
                 '매도5차선호가',
                 '매도5차선잔량',
                 '매도6차선호가',
                 '매도6차선잔량',
                 '매도7차선호가',
                 '매도7차선잔량',
                 '매도8차선호가',
                 '매도8차선잔량',
                 '매도9차선호가',
                 '매도9차선잔량',
                 '매도10차선호가',
                 '매도10차선잔량'
                 ],
    'OPT10003': ['시간',
                 '현재가',
                 '전일대비',
                 '대비율',
                 '우선매도호가단위',
                 '우선매수호가단위',
                 '체결거래량',
                 'sign',
                 '누적거래량',
                 '누적거래대금',
                 '체결강도',
                 ],
}

In [4]:
class KiwoomAPIWindow(QMainWindow):
    def __init__(self):
        super().__init__()
        self.setWindowTitle("KiwomStock")
        self.setGeometry(300, 300, 200, 200)
        self.kiwoom = QAxWidget("KHOPENAPI.KHOpenAPICtrl.1")
        self.kiwoom.CommConnect()
        self.kiwoom.OnEventConnect.connect(self.event_connect)
        
        # Tran 수신시 이벤트
        self.kiwoom.OnReceiveTrData.connect(self.receive_trdata)
        
        
        # TextEdit 생성
        self.text_edit = QTextEdit(self)
        self.text_edit.setGeometry(25, 0, 150, 50)
        self.text_edit.setEnabled(False)
        
        # 버튼 생성
        self.btn1 = QPushButton("실행", self)
        self.btn1.setStyleSheet("color: green;"
                               "background-color: #7FFFD4")
        self.btn1.setCheckable(True)
        self.btn1.toggled.connect(self.btn1_toggled)
        self.btn1.setGeometry(50, 120, 100, 30)
        
        # 데이터 저장용
        self.data = []
        
    # 버튼 동작 설정
    @pyqtSlot(bool)    
    def btn1_toggled(self, state):
        state = !state
        if state:
            self.btn1.setStyleSheet("color: red;"
                             "border-style: solid;"
                             "border-width: 2px;"
                             "border-color: #FA8072;"
                             "border-radius: 3px")
            self.btn1.setText("조회중")
            self.Bids_and_Asks()
        else:
            self.btn1.setStyleSheet("color: green;"
                               "background-color: #7FFFD4")
            self.btn1.setText("실행")
        i = 0
        while state:
            if i == 15000:
                self.btn1.setStyleSheet("color: blue;"
                              "background-color: #87CEFA;"
                              "border-width: 3px;"
                              "border-color: #1E90FF")
                self.btn1.setText("완료")
                self.text_edit.append("Gathering Complete")
                self.Save_As_Json()
                break
            self.Bids_and_Asks()
            self.sleep(1)
            i+=1
        return
    
    def closeEvent(self, event):
        reply = QMessageBox.question(self, 'Window Close', 'Are you sure you want to close the window?',
                                     QMessageBox.Yes | QMessageBox.No, QMessageBox.No)
        if reply == QMessageBox.Yes:
            event.accept()
            print('Window closed')
        else:
            event.ignore()
        
    def sleep(self, n):
        QtTest.QTest.qWait(n*1000)
        return
    
    # 호가 정보 불러오기
    def Bids_and_Asks(self):
        # 파라미터 세팅
        self.kiwoom.SetInputValue("종목코드", COM_CODE)
        self.kiwoom.SetInputValue("수정주가구분", "0")
        # sRQName, sTrCode, nPrevNext, sScreenNo
        res = self.kiwoom.CommRqData("OPT10004_호가조회", "OPT10004", 0, "10004")
        if res == 0:
            print('호가 요청 성공!!' + str(res))
        else:
            print('호가 요청 실패!!' + str(res))
        return
    
    
    # json파일로 저장하기
    def Save_As_Json(self):
        temp_int = 0
        file_data = dict()
        file_data[COM_DATE] = self.data
        file_path = "C:\\Users\\kog07\\Desktop\\STOCK_ANALYSIS\\Data\\{name}.json".format(name = SEARCH_DATE)
        while os.path.isfile(file_path):
            temp_int += 1
            file_path = "C:\\Users\\kog07\\Desktop\\STOCK_ANALYSIS\\Data\\{name}.json".format(name = SEARCH_DATE+"_"+str(temp_int))
            print(file_path)
        with open(file_path, 'w', encoding="utf-8") as make_file:
            json.dump(file_data, make_file, ensure_ascii=False, indent="\t")
        self.data = []
        return
    
    
    # CallBack 함수
    def event_connect(self, nErrCode):
        if nErrCode == 0:
            self.text_edit.append("Login Success")
        return
    
    
    # CallBack 함수
    def receive_trdata(self, sScrNo, sRQName, sTrCode, sRecordName, sPreNext, nDataLength, sErrorCode, sMessage, sSplmMsg):
        if sRQName == "OPT10004_호가조회":
            temp_dict = dict()
            dataCount = self.kiwoom.GetRepeatCnt(sTrCode, sRQName)
            print('총 데이터 수 : ', dataCount)
            for dataIdx in range(dataCount):
                inputVal = output_list['OPT10004']
                for i in inputVal:
                    temp_dict[i] = self.kiwoom.GetCommData(sTrCode, sRQName, dataIdx, i)
                self.data.append(temp_dict)
                print("{time}_저장완료".format(time = temp_dict[inputVal[0]]))

In [5]:
if __name__ == "__main__":
    app = QApplication(sys.argv)
    kaWindow = KiwoomAPIWindow()
    kaWindow.show()
    app.exec_()

Window closed
